In [1]:
library('gdata')


gdata: read.xls support for 'XLS' (Excel 97-2004) files ENABLED.

gdata: read.xls support for 'XLSX' (Excel 2007+) files ENABLED.

Attaching package: ‘gdata’

The following object is masked from ‘package:stats’:

    nobs

The following object is masked from ‘package:utils’:

    object.size

The following object is masked from ‘package:base’:

    startsWith



In [2]:
files_to_read <- list.files(path = '../fichiers_csv/meteo_data',full.names = TRUE)


In [3]:
df_energie <- read.xls('../fichiers_csv/energie.xls')
is.data.frame(df_energie)

[1] TRUE

In [4]:

liste_meteo <- lapply(files_to_read,function(i){
    list_csv <- read.xls(i)
})

In [5]:
df_meteo <- lapply(liste_meteo,function(i){
    a <- data.frame(i,stringsAsFactors = TRUE)
station <- levels(a[1,'X.1'])[13]
mat <- as.matrix(a)
mat <- mat[-c(1,2,3,4,5,6,7),]
colnames(mat) <- mat[1,]
mat <- mat[-1,]
colnames(mat)[1] <- 'Annee'
nb_row <- length(mat[,'Annee'])
mat<- cbind(rep(station,nb_row),mat)
colnames(mat)[1] <- 'Station'
as.data.frame(mat)
})

In [10]:
df_meteo[5]

Warning message in FUN(X[[i]], ...):
“la chaîne de caractères entrée 4 est incorrecte dans cet environnement linguistique”Warning message in FUN(X[[i]], ...):
“la chaîne de caractères entrée 10 est incorrecte dans cet environnement linguistique”Warning message in FUN(X[[i]], ...):
“la chaîne de caractères entrée 14 est incorrecte dans cet environnement linguistique”Warning message in FUN(X[[i]], ...):
“la chaîne de caractères entrée 4 est incorrecte dans cet environnement linguistique”Warning message in FUN(X[[i]], ...):
“la chaîne de caractères entrée 10 est incorrecte dans cet environnement linguistique”Warning message in FUN(X[[i]], ...):
“la chaîne de caractères entrée 14 est incorrecte dans cet environnement linguistique”Warning message in FUN(X[[i]], ...):
“la chaîne de caractères entrée 4 est incorrecte dans cet environnement linguistique”Warning message in FUN(X[[i]], ...):
“la chaîne de caractères entrée 10 est incorrecte dans cet environnement linguistique”Warning message in 

Station,Annee,JAN,FV,MAR,AVR,MAI,JUN,JUI,AO,SEP,OCT,NOV,DC,Total
AURILLAC (15),2010,555.8,446.4,393.4,230.9,218.6,86.7,12.8,55.6,119.9,237.9,379.2,514.4,3251.4
AURILLAC (15),2011,472.2,371.5,335,169,104.3,93.9,80.1,36.8,53.2,193.4,238.6,401.8,2549.5
AURILLAC (15),2012,462.6,590.2,298.8,304.1,137.4,60.9,56.6,19,94.5,180.8,317.1,443.6,2965.3
AURILLAC (15),2013,526.2,480.3,376.3,282.2,268.3,109.3,7.1,39,95.1,160.5,391.6,415.5,3151.1
AURILLAC (15),2014,402.9,369.5,332.7,219.4,197.5,52,42.9,72.4,60.9,126.8,244,453.6,2574.3
AURILLAC (15),2015,465.3,456.1,329.2,210.3,152,32,11.8,35.6,144.5,234.1,298.9,321.1,2690.7
AURILLAC (15),2016,413.7,386.3,394.3,279.7,182.7,79.5,25.8,22.6,70.5,232.1,329,395.1,2811
AURILLAC (15),2017,532.9,323.7,287.4,265.9,128.6,0,0,0,0,0,0,0,1538.4
